# Let's look at the raw data, specifically the washover stats

In [1]:
# Import packages we need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,shutil

In [2]:
#load the labels
#round 2 labels
#labeler_data = pd.read_csv('../data/tables/labels/Priority_1604952200293.csv')
#round 3 labels
#labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1613057474309.csv')
#round 4 labels
#labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1618341223365.csv')
#round 5 labels
labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1629556104483.csv')
#round5.5 labels
labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1631025488443.csv')

## Get data into shape for release

In [3]:
# make data into final zenodo form
releaseData = labeler_data
releaseData = releaseData.drop(['_id','catalogId', 'archiveId','date','catalog','additionalComments'], axis = 1)
releaseData = releaseData.drop(['imageId'], axis = 1)
releaseData = releaseData.rename({'water': 'allWater'}, axis=1)

releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,NaN,NaN,NaN,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,NaN,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,NaN,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,NaN,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,NaN,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline


In [4]:
# replace NaN in devType, washoverType, dmgType,
values = {'allWater':'False', 'devType': 'undeveloped', 'washoverType': 'noWashover', 'dmgType': 'noDamage'}

releaseData = releaseData.fillna(value=values)
releaseData

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,False,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,False,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,False,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline
...,...,...,...,...,...,...,...,...,...
8156,5f4c002a9cdb9b001675a8e2,Sally6,S26561809.jpg,False,developed,noWashover,noDamage,NaN,inland
8157,5f4c002a9cdb9b001675a8e2,Sally6,P26365345.jpg,False,developed,washover,noDamage,overwash,"sandyCoastline,marsh,inland"
8158,5f4c002a9cdb9b001675a8e2,Sally6,P26365549.jpg,False,developed,noWashover,noDamage,NaN,"marsh,estuary"
8159,5f4c002a9cdb9b001675a8e2,Sally6,P26553233.jpg,False,developed,washover,noDamage,overwash,"marsh,estuary"


In [5]:
releaseData.shape

(8161, 9)

In [6]:
Threshold = releaseData[["userId","archive","image"]].groupby(['userId', 'archive']).count().reset_index()
#Threshold = Threshold[Threshold["image"] == 100].reset_index()
Threshold

,userId,archive,image
0,5f4c002a9cdb9b001675a8e2,Barry5,100
1,5f4c002a9cdb9b001675a8e2,Barry6,50
2,5f4c002a9cdb9b001675a8e2,Delta5,100
3,5f4c002a9cdb9b001675a8e2,Delta6,50
4,5f4c002a9cdb9b001675a8e2,Dorian5,100
...,...,...,...
91,6065fb60693eea0016eea0c1,Isaias4,100
92,609ea7ef4a006500165c5de4,Dorian5,100
93,609ea7ef4a006500165c5de4,Isaias5,100
94,609ea7ef4a006500165c5de4,Zeta5,100


In [7]:
#add a true column
Threshold.rename(columns = {"image": "total"}, inplace=True)

#join Threshold with releaseData
releaseData = releaseData.merge(Threshold, left_on = ['userId', 'archive'], right_on = ['userId', 'archive'])

# remove rows where total <100
releaseData = releaseData[releaseData["total"] >= 50]
releaseData

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,5f4c002a9cdb9b001675a8e2,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,5f4c002a9cdb9b001675a8e2,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,5f4c002a9cdb9b001675a8e2,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
...,...,...,...,...,...,...,...,...,...,...
8156,5f4c002a9cdb9b001675a8e2,Sally6,S26561809.jpg,False,developed,noWashover,noDamage,NaN,inland,50
8157,5f4c002a9cdb9b001675a8e2,Sally6,P26365345.jpg,False,developed,washover,noDamage,overwash,"sandyCoastline,marsh,inland",50
8158,5f4c002a9cdb9b001675a8e2,Sally6,P26365549.jpg,False,developed,noWashover,noDamage,NaN,"marsh,estuary",50
8159,5f4c002a9cdb9b001675a8e2,Sally6,P26553233.jpg,False,developed,washover,noDamage,overwash,"marsh,estuary",50


In [8]:
#replace user IDs with `User 1`, etc...
releaseData.shape
releaseData.userId.unique()

array(['5f4c002a9cdb9b001675a8e2', '5f77871024f95e0018305349',
       '5f6635628cb36a001698427b', '5f7af56443360b001790063d',
       '5f7c8c4287bff50017610126', '5f89a4957c0f40001652c05d',
       '5f89c25b67d0e30016a5e70a', '5fa4542fec49d20016f772a8',
       '5fbc7ccabf106c0017b864a0', '5fc7dc859f59bc0016211439',
       '5f8f5f59e4dac20017f9070e', '600994de5eff87001694a117',
       '5ff24c3448a4570013d1a9ed', '6050c31a1bd6510018fa2a87',
       '6051072d523d7200169cc5e5', '6052b1a5f441e80013ac154e',
       '6054a37b79f962001798e2eb', '6054b60503d3fc0013d43782',
       '6064ef18340146001681bf0c', '6065fb60693eea0016eea0c1',
       '609ea7ef4a006500165c5de4', '60ac0f1d20620600172dcfd9',
       '5fc94f320dc93600168cf242'], dtype=object)

In [9]:
releaseData['userId'] = pd.factorize(releaseData.userId)[0] + 1
releaseData.userId.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23])

In [10]:
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,1,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,1,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,1,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,1,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,1,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100


In [11]:
releaseData.archive.unique()

array(['Florence', 'Michael', 'Isaias', 'MichaelWash3', 'Isaias3',
       'Florence3', 'FlorenceWash3', 'Michael3', 'IsaiasWash3',
       'Michael4', 'MichaelWash4', 'Florence4', 'IsaiasWash4', 'Isaias4',
       'FlorenceWash4', 'Barry5', 'Delta5', 'Dorian5', 'Gordon5',
       'Laura5', 'Sally5', 'Zeta5', 'Isaias5', 'Florence5', 'Michael5',
       'Barry6', 'Zeta6', 'Delta6', 'Laura6', 'Sally6'], dtype=object)

In [12]:
releaseData['LabelerArchive'] = releaseData['archive']

#remove 'Wash'
releaseData['archive'] = releaseData['archive'].str.replace("Wash", "")

#remove numbers
releaseData['archive'] = releaseData['archive'].map(lambda x: x.rstrip('3456'))

#lowercase
releaseData['archive'] = releaseData['archive'].str.lower()

#rename archive to storm_id
releaseData = releaseData.rename({'archive': 'storm_id'}, axis=1)  

print(releaseData.storm_id.unique())

releaseData.head()

['florence' 'michael' 'isaias' 'barry' 'delta' 'dorian' 'gordon' 'laura'
 'sally' 'zeta']


,userId,storm_id,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total,LabelerArchive
0,1,florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100,Florence
1,1,florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,Florence
2,1,florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,Florence
3,1,florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,Florence
4,1,florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,Florence


In [13]:
# now to link with big dataframe

#read in the master DF
df = pd.read_csv('../data/tables/MasterDF_08262021_R6.csv')
df.head()

,Filename,Prediction1,Prediction2,Prediction3,Prediction4,archive,image,date,labeled,ll_lat,ll_lon,lr_lat,lr_lon,size,storm_id,ul_lat,ul_lon,ur_lat,ur_lon,Prediction5
0,Barry/20190716a_jpgs/jpgs/C19748935.jpg,0.093053,0.458411,0.679300,0.274590,20190716a_jpgs,C19748935.jpg,2019/07/16,NaN,29.508246,-92.321909,29.501893,-92.311318,4875221,barry,29.514684,-92.317575,29.509719,-92.306144,0.021257
1,Barry/20190716a_jpgs/jpgs/C19748941.jpg,0.002945,0.449371,0.456580,0.237515,20190716a_jpgs,C19748941.jpg,2019/07/16,NaN,29.511797,-92.319636,29.506580,-92.308422,5842249,barry,29.518644,-92.316049,29.514679,-92.304272,0.043771
2,Barry/20190716a_jpgs/jpgs/C19748947.jpg,0.002490,0.413135,0.469627,0.226322,20190716a_jpgs,C19748947.jpg,2019/07/16,NaN,29.515591,-92.317814,29.511428,-92.306184,6061687,barry,29.522755,-92.314927,29.519666,-92.302932,0.052162
3,Barry/20190716a_jpgs/jpgs/C19748953.jpg,0.020509,0.361904,0.444304,0.202061,20190716a_jpgs,C19748953.jpg,2019/07/16,NaN,29.519760,-92.316963,29.516382,-92.305242,5873595,barry,29.527209,-92.314396,29.524401,-92.302525,0.065795
4,Barry/20190716a_jpgs/jpgs/C19748959.jpg,0.043591,0.347595,0.443178,0.191845,20190716a_jpgs,C19748959.jpg,2019/07/16,NaN,29.523765,-92.315955,29.521040,-92.304028,6159050,barry,29.531420,-92.313768,29.529043,-92.301852,0.061327


In [14]:
# selecting rows based on being labeled
L_df = df[df['labeled'].notnull()]

#select based on the labeled round
#L_df = df[df['labeled'] == 5]

###### image and filename

#Any Duplicates?, should be false via: 
#https://stackoverflow.com/questions/50242968/check-for-duplicate-values-in-pandas-dataframe-column

L_df.duplicated(subset=['image','storm_id']).any()

False

In [15]:
#combine the storm name and archive into the flight tag
L_df["NOAA_flight"] = L_df["storm_id"] + "/" + L_df['archive']

#subset dataframe into just the image, storm id and NOAA flight info
L_df = L_df[['image','storm_id','NOAA_flight']]

<ipython-input-15-ddd9d8bad82a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L_df["NOAA_flight"] = L_df["storm_id"] + "/" + L_df['archive']


In [16]:
#merge to prediction dataframe
releaseData = pd.merge(releaseData, L_df, on=["image", "storm_id"], how = "left")
releaseData.head()

,userId,storm_id,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total,LabelerArchive,NOAA_flight
0,1,florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100,Florence,florence/20180917a_jpgs
1,1,florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,Florence,florence/20180917a_jpgs
2,1,florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,Florence,florence/20180917a_jpgs
3,1,florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100,Florence,florence/20180917a_jpgs
4,1,florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100,Florence,florence/20180917a_jpgs


In [17]:
releaseData.shape

(8087, 12)

In [18]:
#save release data
releaseData.to_csv('../data/tables/labelerdata/ReleaseData09072021.csv',na_rep='NaN')